<a href="https://colab.research.google.com/github/kshitizkool/Groq/blob/main/Text_to_SQL_model_using_Groq_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

from getpass import getpass

# Prompt user to enter their API keys securely
groq_api_key = getpass("Please enter your Groq API key: ")


# Set environment variables
os.environ['GROQ_API_KEY'] = groq_api_key

print("API keys have been set.")

Please enter your Groq API key: ··········
API keys have been set.


In [ ]:
pip install groq duckdb sqlparse numpy pandas tabulate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
import os
from groq import Groq
import json
import duckdb
import sqlparse
import numpy as np
import pandas as pd

In [ ]:
def chat_with_groq(client, prompt, model, response_format):
  completion = client.chat.completions.create(
  model=model,
  messages=[
      {
          "role": "user",
          "content": prompt
      }],
  response_format=response_format
  )
  return completion.choices[0].message.content

In [ ]:
def execute_duckdb_query(query):
  original_cwd = os.getcwd()
  os.chdir('data')

  try:
      conn = duckdb.connect(database=':memory:', read_only=False)
      query_result = conn.execute(query).fetchdf().reset_index(drop=True)
  finally:
      os.chdir(original_cwd)

  return query_result

In [ ]:
def get_summarization(client, user_question, df, model):
    """
    This function generates a summarization prompt based on the user's question and the resulting data.
    It then sends this summarization prompt to the Groq API and retrieves the AI's response.

    Parameters:
    client (Groqcloud): The Groq API client.
    user_question (str): The user's question.
    df (DataFrame): The DataFrame resulting from the SQL query.
    model (str): The AI model to use for the response.

    Returns:
    str: The content of the AI's response to the summarization prompt.
    """
    prompt = '''
      A user asked the following question pertaining to local database tables:

      {user_question}

      To answer the question, a dataframe was returned:

      Dataframe:
      {df}

    In a few sentences, summarize the data in the table as it pertains to the original user question. Avoid qualifiers like "based on the data" and do not comment on the structure or metadata of the table itself
  '''.format(user_question = user_question, df = df)

    # Response format is set to 'None'
    return chat_with_groq(client,prompt,model,None)


In [ ]:
model = "llama3-70b-8192"

# Get the Groq API key and create a Groq client
groq_api_key = os.getenv('GROQ_API_KEY')
client = Groq(
  api_key=groq_api_key
)

In [ ]:
print("Welcome to the DuckDB Query Generator!")
print("You can ask questions about the data files.")

Welcome to the DuckDB Query Generator!
You can ask questions about the data files.


In [ ]:
with open('/content/data/base_prompt.txt', 'r') as file:
  base_prompt = file.read()

while True:
  # Get the user's question
  user_question = input("Ask a question: ")

  if user_question:
      # Generate the full prompt for the AI
      full_prompt = base_prompt.format(user_question=user_question)

      # Get the AI's response. Call with '{"type": "json_object"}' to use JSON mode
      llm_response = chat_with_groq(client, full_prompt, model, {"type": "json_object"})

      result_json = json.loads(llm_response)
      if 'sql' in result_json:
          sql_query = result_json['sql']
          results_df = execute_duckdb_query(sql_query)

          formatted_sql_query = sqlparse.format(sql_query, reindent=True, keyword_case='upper')

          print("```sql\n" + formatted_sql_query + "\n```")
          print(results_df.to_markdown(index=False))

          summarization = get_summarization(client,user_question,results_df,model)
          print(summarization)
      elif 'error' in result_json:
          print("ERROR:", 'Could not generate valid SQL for this question')
          print(result_json['error'])

```sql
SELECT Name
FROM pokemon.csv AS pokemon
WHERE Attack > 50
```
| Name                      |
|:--------------------------|
| Ivysaur                   |
| Venusaur                  |
| VenusaurMega Venusaur     |
| Charmander                |
| Charmeleon                |
| Charizard                 |
| CharizardMega Charizard X |
| CharizardMega Charizard Y |
| Wartortle                 |
| Blastoise                 |
| BlastoiseMega Blastoise   |
| Beedrill                  |
| BeedrillMega Beedrill     |
| Pidgeotto                 |
| Pidgeot                   |
| PidgeotMega Pidgeot       |
| Rattata                   |
| Raticate                  |
| Spearow                   |
| Fearow                    |
| Ekans                     |
| Arbok                     |
| Pikachu                   |
| Raichu                    |
| Sandshrew                 |
| Sandslash                 |
| Nidorina                  |
| Nidoqueen                 |
| Nidoran?                  |
|

KeyboardInterrupt: Interrupted by user